# Analiza podatkov

In [1]:
import pandas as pd

filmi = pd.read_csv("filmi.csv", index_col="id")

In [2]:
zanri = pd.read_csv("filmi_zanri.csv")

In [8]:
def koren_besede(beseda):
    if len(beseda) <= 2 or beseda == "the" or beseda == "and":
        return ""

    if beseda[-1] in "sye":
        return beseda[:-1]

    if beseda[-3:] == "ing":
        return beseda[:-3]

    if beseda[-2:] == "ed":
        return beseda[:-2]
    
    return beseda


def koreni_opisa(opis):
    opis = opis.lower()
    opis = "".join([crka for crka in opis if crka.isalpha() or crka.isspace()])

    besede = opis.split(" ")

    koreni = [koren_besede(beseda) for beseda in besede if koren_besede(beseda) != ""]

    return pd.Series(koreni)



koreni_opisa("Alien spaceship files to another planet.")


0        alien
1    spaceship
2         file
3      another
4       planet
dtype: object

In [10]:
koren = filmi["opis"].apply(koreni_opisa)

In [17]:
filmi_besede = koren.stack().reset_index("id").rename(columns={0:"koren"})

In [19]:
zanri_koreni = pd.merge(filmi_besede, zanri, left_on="id", right_on="id_filma")[["koren", "zanr"]]
zanri_koreni

koren                   zanr
0      screenwriter            Dark Comedy
1      screenwriter              Film Noir
2      screenwriter    Psychological Drama
3      screenwriter          Showbiz Drama
4      screenwriter                Tragedy
...             ...                    ...
91289         basel            Documentary
91290       israeli  Political Documentary
91291       israeli            Documentary
91292    journalist  Political Documentary
91293    journalist            Documentary

[91294 rows x 2 columns]

In [26]:
koreni_verjetnosti = pd.crosstab(zanri_koreni["koren"], zanri_koreni["zanr"]) / zanri.groupby("zanr").size() + 0.001

In [23]:
verjetnosti_zanrov = zanri.groupby("zanr").size() / len(filmi)
verjetnosti_zanrov

zanr
Action             0.083721
Action Epic        0.026977
Adult Animation    0.004651
Adventure          0.150698
Adventure Epic     0.034419
                     ...   
Western            0.025116
Western Epic       0.008372
Whodunnit          0.009302
Workplace Drama    0.011163
Zombie Horror      0.000930
Length: 161, dtype: float64

In [24]:
opis = "Alien spaceship files to another planet."

In [32]:
def zanr_opisa(opis):
  return (koreni_verjetnosti[koreni_verjetnosti.index.isin(koreni_opisa(opis))].prod() * verjetnosti_zanrov).sort_values(ascending=False).head(5)

In [33]:
zanr_opisa(opis)

zanr
Space Sci-Fi     6.356848e-08
Sci-Fi           1.843009e-08
Sci-Fi Epic      1.597424e-08
Adventure        4.722130e-09
Coming-of-Age    4.307442e-10
dtype: float64

In [35]:
zanr_opisa("Young student of math finds programming anoying and searches for the meaning of life.")

zanr
Coming-of-Age    1.049019e-10
Comedy           4.214424e-11
Drama            3.329717e-11
Music            1.121251e-11
Romance          7.090614e-12
dtype: float64